In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import sys 
import os 
import pyspark.sql.functions as f 
sys.path.append(os.path.abspath('../..'))

In [0]:
from data_loaders.data_load_autoloader import load_raw_data_with_schema_evolution
from data_writers.write_data import write_bronze_data

In [0]:
CATALOG = "pei"
STORAGE_BASE = "abfss://retail@peiretailstorageaccount.dfs.core.windows.net/landing"

In [0]:
order_load_options = {
    "badRecordsPath": f"{STORAGE_BASE}/orders",
    "cloudFiles.schemaLocation": f"/Volumes/{CATALOG}/default/checkpoint_volume/raw_orders_schema_checkpoint"
}

In [0]:
try:
  # read orders
  df_orders = load_raw_data_with_schema_evolution(spark_session=spark, 
                                          source_path=f"{STORAGE_BASE}/orders/*.json",
                                          file_format="json",
                                          additional_options=order_load_options)

  # write orders
  write_bronze_data(
    df=df_orders,
    target_table=f"{CATALOG}.bronze.raw_orders",
    checkpoint_location=f"/Volumes/{CATALOG}/default/checkpoint_volume/raw_orders_file_checkpoint", 
    is_streaming=True,
    file_source_path=None,
    trigger={"availableNow": True}
    )
          
except Exception as e: 
  print(f"FAILED: Orders Ingestion. Error: {str(e)}")               
        